In [5]:
import torch

from datasets import LabelledPointCloudDataset
from models import LabelledPointNetAE, PointNet, Decoder, Segmenter

import json
import os
from pathlib import Path

In [6]:
import numpy as np
import pandas as pd
import torch

import os

class LabelledPointCloudDataset(torch.utils.data.Dataset):
    def __init__(self, dir, c, radius=100, max_points=None):
        fps = []
        for root, dirs, files in os.walk(dir):
            fps.extend([os.path.join(dir, file) for file in files])
            length = len(files)
            break
        self.fps = fps
        self.length = length
        self.c = c
        self.radius = radius
        self.max_points = max_points
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        df = pd.read_csv(self.fps[idx], header=0, names=['x', 'y', 'z', "label"])
        if self.max_points:
            df = df.sample(self.max_points)
        centroid = df[['x', 'y', 'z']].mean().values
        points = torch.tensor((df[['x', 'y', 'z']].values - centroid) / self.radius, dtype=torch.float32)
        labels_df = df["label"].values
        labels = torch.zeros(len(df), self.c, dtype=torch.float32)
        labels[range(len(labels_df)), labels_df] = 1
        tensor = torch.cat((points, labels), dim=1)
        tensor = tensor.transpose(0, 1)
        return '.'.join(self.fps[idx].split('/')[-1].split('.')[:-1]), tensor

    def getN(self):
        return self.__getitem__(0).size()[1]

class LabelledPointNetAE(torch.nn.Module):
    def __init__(self, n, c):
        super(LabelledPointNetAE, self).__init__()
        self.n = n

        self.encoder1 = PointNet()
        self.encoder2 = PointNet()
        self.decoder = Decoder(n)
        self.segmenter = Segmenter(c)

    def forward(self, x):
        batch_size = x.size()[0]
        x1, _ = self.encoder1(x)
        # points = self.decoder(x1)
        # global_feature, local_feature = self.encoder2(points)
        # x2 = torch.cat((global_feature.view(-1, 1024, 1).repeat(1, 1, self.n), local_feature), dim=1)
        # labels = self.segmenter(x2)
        return x1

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
n = 16384
c = 19

dataset = LabelledPointCloudDataset("../data/beautyworld-train/", c, 100, max_points=n)
loader = torch.utils.data.DataLoader(dataset, 1, shuffle=False)

model = LabelledPointNetAE(n, c)
model.load_state_dict(torch.load(
    "./runs/hpc-2025-03-04/best-model-state-dict.pt", weights_only=True, map_location=torch.device(device)
))
model.to(device)
model.eval()

LabelledPointNetAE(
  (encoder1): PointNet(
    (tnet1): TNet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (tnet2): TNet(
      (conv1): Conv1d(64, 64, kernel_s

In [8]:
i = 0
for batch in loader:
    name, x = batch
    encoding = model(x[:, :3, :])
    with open(f"data/beautyworld-encodings/{name[0]}.json", 'w') as fp:
        json.dump(encoding[0].cpu().tolist(), fp)
    i += 1